This notebook demonstrate how to perform various analytics on tweeter tweets data

Install dependencies

In [1]:
# !pip install dash-bootstrap-components
# !pip install dash-renderer

Import required libraries

In [15]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import plotly.graph_objects as go
import dash_html_components as html
from dash.dependencies import Input, Output,State
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from email.utils import parsedate_tz

import pandas as pd
from geopandas.tools import geocode
import geopandas
from urllib.request import urlopen
import json

# Tweet extraction and processing
import os
import re
import time
import tweepy as tw
from textblob import TextBlob

In [3]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [4]:
# Connection credentials
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'
# End connection credentials

# Define the AUTHHandler
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)
#End AUTHHandler

In [5]:
def extract_tweets(search_words,no_of_tweets):
    start_time = time.time()
    # Define date and search Keyword
    date_since =pd.to_datetime('today').strftime("%Y-%m-%d")
    #Define the cursor
    tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(no_of_tweets)
    # Clean text
    text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
    # Create DataFrame 
    users_locs = [[tweet.user.screen_name,tweet.user.name,tweet.user.verified,
             tweet.user.followers_count,tweet.user.friends_count,tweet.user.listed_count,
             tweet.retweet_count,tweet.favorite_count,tweet.retweeted,tweet.entities,
             tweet.user.favourites_count,
             tweet.user.location,tweet.created_at,tweet.text,
             re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text))),
             TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[0],
             TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[1]
                  ] for tweet in tweets]
    cols=columns=['screen_name','name','user_verification','followers_count','friends_count',
                  'listed_count','retweet_count','favorite_count','retweeted','entities','favourites_count',
                  'location','created_at','text','clean_text','sentiment_polarity','sentiment_subjectivity']
    df = pd.DataFrame(data=users_locs, columns=cols)
    df["sentiment_polarity_color"] = np.where(df["sentiment_polarity"]<0, 'red', 'green')
    print("--- %s seconds ---" % (time.time() - start_time))
    return df

In [7]:
search_words = "#Covid"
no_of_tweets=15
df_online=extract_tweets(search_words,no_of_tweets)

--- 7.694837331771851 seconds ---


In [8]:
df_offline=pd.read_csv("../datasets/tweets.csv")

In [9]:
df=pd.DataFrame()
if df_online.shape[0]==0:
    df=df_offline
else:
    df=df_online.shape[0]

In [10]:
df.head()

,Unnamed: 0,screen_name,name,user_verification,followers_count,friends_count,listed_count,retweet_count,favorite_count,retweeted,entities,favourites_count,location,created_at,text,clean_text,sentiment_polarity,sentiment_subjectivity,sentiment_polarity_color
0,0,nat___price,natalie price,False,62,732,0,2,0,False,"{'hashtags': [{'text': 'covid', 'indices': [45...",13671,"Yorkshire and The Humber, England",2021-01-30 09:09:05,RT @NickTriggle: The devastating toll of the #...,The devastating toll of the covid pandemic on...,-0.550000,0.600000,red
1,1,libertad717,Punto,False,1084,4760,468,11,0,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",208,NaN,2021-01-30 09:09:00,RT @NATOBrazeB: Great to meet online @UN’s Und...,Great to meet online Undersecretary for Glob...,0.400000,0.375000,green
2,2,marylouisepearc,marylouise lady of leisure,False,124,244,3,7,0,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",57448,NaN,2021-01-30 09:08:58,RT @SiCarswell: That screeching noise to be he...,That screeching noise to be heard is the EU C...,-0.008333,0.550000,red
3,3,JhSalford,JHSalford,False,177,177,0,336,0,False,"{'hashtags': [{'text': 'COVID', 'indices': [76...",435,"Salford, England",2021-01-30 09:08:56,RT @GabrielScally: Don't be silly. The UK gove...,Don't be silly. The UK government has complet...,-0.077778,0.751389,red
4,4,SarahEdmondsPhD,Fear is the MindKiller,False,817,1861,9,387,0,False,"{'hashtags': [{'text': 'Covid', 'indices': [12...",53062,"Flagstaff, AZ",2021-01-30 09:08:49,RT @AlexBerenson: So after four weeks more ser...,So after four weeks more serious adverse even...,0.083333,0.583333,green


In [ ]:
df.columns

Number of tweets

In [ ]:
number_of_tweets=df['name'].count()
number_of_tweets

Likes

In [ ]:
favourites_count=round(df['favourites_count'].sum()/1000000,2)
favourites_count

Users Tweet

In [ ]:
users_count=df['name'].nunique()
users_count

Sentiment polarity

In [ ]:
sentiment_polarity=round(df['sentiment_polarity'].mean(),2)
sentiment_polarity

Sentiment polarity and subjectivity line graph

In [ ]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=df['created_at'], y=df['sentiment_polarity'], name='Polarity',line = dict(color='skyblue'))) 
fig.update_layout(dict(autosize=True,margin=dict(t=0,b=0,l=0,r=0),xaxis=dict(title = 'Period', ticklen=2, zeroline=False)))
# fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),title={'text': "Amazon",'y':0.75,'x':0.4,'xanchor': 'center','yanchor': 'middle'})

fig.show()

Sentiment polarity by region and users

In [ ]:
df=df.dropna(subset=['name', 'location'], how='any')
regional_avg_sentiment_df=pd.DataFrame(df.groupby(['location'],as_index=False)['sentiment_polarity'].mean()).head(50)
regional_avg_sentiment_df=regional_avg_sentiment_df[(regional_avg_sentiment_df['sentiment_polarity'] != 0.000)]
user_avg_sentiment_df=pd.DataFrame(df.groupby(['name'],as_index=False)['sentiment_polarity'].mean()).head(50)
user_avg_sentiment_df=user_avg_sentiment_df[(user_avg_sentiment_df['sentiment_polarity'] != 0.000)]
# dff=regional_avg_sentiment_df[regional_avg_sentiment_df['location'].isin([region])]

fig = make_subplots(rows=1, cols=2,shared_xaxes=False,shared_yaxes=True,vertical_spacing=0.03,specs=[[{"type": "bar"},{"type": "bar"}]],
    column_width=[50, 50],horizontal_spacing=0.015)

fig.add_trace(go.Bar(name='Region',x=regional_avg_sentiment_df['location'].str[:20],y=regional_avg_sentiment_df['sentiment_polarity'],marker=dict(color="skyblue"), showlegend=True),row=1, col=1)
fig.add_trace(go.Bar(name='User',x=user_avg_sentiment_df["name"].str[:20],y=user_avg_sentiment_df["sentiment_polarity"], marker=dict(color="teal"), showlegend=True),row=1, col=2)
fig.update_layout(dict(autosize=True,margin=dict(t=0,b=0,l=0,r=0),xaxis=dict(ticklen=2, zeroline=False),legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),))

fig.show()

Using conditional filter in pandas

In [ ]:
df[(df['created_at'] > '2021-01-28') & (df['created_at'] <= '2021-01-29')].head()

Get max and min value based on date

In [ ]:
max(df['created_at'])
min(df['created_at'])

In [ ]:
df=df[~df['location'].str.isnumeric()==True]

Sentiment Geo

geocode the location to enrich with coordinates

In [ ]:
cities_df = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
geo_df=df[df['location'].isin(cities_df['name'])]

In [ ]:
def custom_geocoder(address):
    try:
        dataframe = geocode(address.strip() , provider="nominatim" , user_agent = 'csc_user_ht')
        point = dataframe.geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x})
    except:
        pass

In [ ]:
geo_df[['latitude' , 'longitude']]= geo_df.iloc[:,12].apply( lambda x: custom_geocoder(x))

In [ ]:
geo_df.head()

In [ ]:
geo_df['normalized_sentiment']="Location : "+geo_df['location']+ " <br>Sentiment : "+geo_df['sentiment_polarity'].astype(str)

In [ ]:
px.set_mapbox_access_token(open("../assets/mapboxtoken.mapbox_token").read())
fig = px.scatter_mapbox(geo_df,
                        lat=geo_df.latitude,
                        lon=geo_df.longitude,
                        size=geo_df['sentiment_polarity']+1,
#                         hover_name=df["normalized_sentiment"],
                        color="sentiment_polarity",
                        text=geo_df['location'],
#                         size_max  =20,
                        mapbox_style ='carto-positron',
                        labels  =geo_df['location'],
                        
                        zoom=1)
fig.show()

Choropleth maps

Load geojson

In [ ]:
with open('../datasets/countries.geojson') as response:
    country_geojson = json.load(response)

Prepare data geo- sentiment data

In [ ]:
# polarity
sentiment_polarity_geo_data_df=df.dropna(subset=['name', 'location'], how='any')
sentiment_polarity_geo_data_df=pd.DataFrame(sentiment_polarity_geo_data_df.groupby(['location'],as_index=False)['sentiment_polarity'].mean())
sentiment_polarity_geo_data_df.columns = ['ADMIN', 'sentiment_polarity']
# subjectivity
sentiment_subjectivity_geo_data_df=df.dropna(subset=['name', 'location'], how='any')
sentiment_subjectivity_geo_data_df=pd.DataFrame(sentiment_subjectivity_geo_data_df.groupby(['location'],as_index=False)['sentiment_subjectivity'].mean())
sentiment_subjectivity_geo_data_df.columns = ['ADMIN', 'sentiment_subjectivity']

Merge geopandas and sentiment data

In [ ]:
# ploarity
sentiment_polarity_geo_df = geopandas.GeoDataFrame.from_features(country_geojson["features"]).merge(sentiment_polarity_geo_data_df, on="ADMIN").set_index("ADMIN")
sentiment_polarity_geo_df.rename(columns={'Admin': 'location'},inplace=True)
# subjectivity
sentiment_subjectivity_geo_df = geopandas.GeoDataFrame.from_features(country_geojson["features"]).merge(sentiment_subjectivity_geo_data_df, on="ADMIN").set_index("ADMIN")
sentiment_subjectivity_geo_df=sentiment_subjectivity_geo_df.rename(columns={'Admin': 'location'})

Save the data for use in dash deployment

In [ ]:
# sentiment_polarity_geo_df.to_csv("../datasets/sentiment_polarity_geo_data.csv")
# sentiment_subjectivity_geo_df.to_csv("../datasets/sentiment_subjectivity_geo_data.csv")

In [ ]:
sentiment_polarity_geo_df.head()

In [ ]:
sentiment_subjectivity_geo_df.head()

Plot map

In [ ]:
# polarity
fig = px.choropleth_mapbox(sentiment_polarity_geo_df,
                           geojson=sentiment_polarity_geo_df.geometry,
                           locations=sentiment_polarity_geo_df.index,
                           color="sentiment_polarity",
#                            center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron",
                           zoom=0.5)
fig.update_layout(dict(autosize=True,margin=dict(t=0,b=0,l=0,r=0),xaxis=dict(ticklen=2, zeroline=False),legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),))

fig.show()

In [ ]:
# subjectivity
fig = px.choropleth_mapbox(sentiment_subjectivity_geo_df,
                           geojson=sentiment_subjectivity_geo_df.geometry,
                           locations=sentiment_subjectivity_geo_df.index,
                           color="sentiment_subjectivity",
#                            center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron",
                           zoom=0.75)
fig.update_layout(dict(autosize=True,margin=dict(t=0,b=0,l=0,r=0),xaxis=dict(ticklen=2, zeroline=False),legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),))

fig.show()

Run Server

In [ ]:
# if __name__ == '__main__':
#     app.run_server(debug=False)